## 在 huggingface 上下载数据

In [ ]:
from datasets import load_dataset 

dataset = load_dataset("roneneldan/TinyStories")

print(dataset)

In [ ]:
import os

os.makedirs("/home/winbeau/Study/1-transformer/datasets/TinyStories", exist_ok=True)

dataset['train'].to_parquet("/home/winbeau/Study/1-transformer/datasets/TinyStories/train.parquet")
dataset['validation'].to_parquet("/home/winbeau/Study/1-transformer/datasets/TinyStories/valid.parquet")

In [ ]:
os.makedirs("/home/winbeau/Study/1-transformer/datasets/TinyStories/txt", exist_ok=True)

with open("/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/train.txt", "w", encoding="utf-8") as f:
    for row in dataset['train']:
        f.write(row["text"].replace("\n", " ") + "\n")
with open("/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/valid.txt", "w", encoding="utf-8") as f:
    for row in dataset['validation']:
        f.write(row["text"].replace("\n", " ") + "\n")

## 为训练数据添加 `<|endoftext|>`
- 将训练集中每个故事添加结束标志，防止故事间跨越，学习全局超长语料
- 依然是全局BPE，但是故事间有 `<|endoftext|>` 作为挡板
- **不是** 每个故事单独训练一个BPE词表再汇总\[这可能会导致出现不同的token编号，模型无法同一使用\]，而是全局统计

In [ ]:
train_raw_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/train.txt"
valid_raw_path = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/valid.txt"

# 查看前几行
def preview_txt(cnt_line, txt_path=train_raw_path): 
    with open(txt_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f): # 行号 和 内容
            if i >= cnt_line:
                break
            print(line.strip())

In [ ]:
preview_txt(2)

In [ ]:
import os 

base_dir = "/home/winbeau/Study/1-transformer/datasets/TinyStories/txt/"

files = [
    ("train.txt", "train_with_eot.txt"), 
    ("valid.txt", "valid_with_eot.txt"),
]

In [ ]:
def add_endoftext(infile: str, outfile: str): 
    """行末添加 <|endoftext|> , 忽略空行 """
    cnt_in, cnt_out = 0, 0
    with open(infile, "r", encoding="utf-8") as fin, open(outfile, "w", encoding="utf=8") as fout: 
        for line in fin: 
            text = line.strip()
            cnt_in += 1
            if text: 
                fout.write(text + "<|endoftext|>\n")
                cnt_out += 1
    print(f"Complete! {cnt_out} / {cnt_in}")

In [ ]:
for fin, fout in files: 
    add_endoftext(
        os.path.join(base_dir, fin),
        os.path.join(base_dir, fout) 
    )
print("All files have added <|endoftext|> and saved!")

In [ ]:
preview_txt(2, os.path.join(base_dir, "train_with_eot.txt")) # 验证是否加入成功